In [ ]:
!pip install pandas
!pip install lxml

In [1]:
import requests 
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import zipfile
import io
import os
import xml.etree.ElementTree as ET
import requests
import time
import random
from tqdm import tqdm

In [2]:
# Alternative mit Weblink
url = "https://www.rechtsprechung-im-internet.de/rii-toc.xml"
response = requests.get(url)
xml_string = response.text


In [3]:
# Inhalte in BeautifulSoup
soup = BeautifulSoup(xml_string, 'xml',)
items = soup.find_all('item')

In [4]:
len(items)

75648

In [5]:
# Hier kann bei Download abbruch bestimmt werden, wo es weitergene soll.
items = items[75561:]

In [6]:
links_list = []

In [7]:
# Begrenzte Ausgabe
for item in items:
    link = item.find('link').text
    
    links_list.append(link)

In [8]:
os.makedirs("Gerichtsurteile", exist_ok=True)

In [9]:
target_dir = "Gerichtsurteile"

In [10]:
def download_and_extract_zip(urls, target_dir):
    # Überprüfe, ob urls tatsächlich eine Liste ist und ob sie nicht leer ist
    if not isinstance(urls, list) or not urls:
        raise ValueError("Die Eingabe muss eine nicht-leere Liste von URLs sein.")
    
    # Stelle sicher, dass der Zielordner existiert oder erstelle ihn
    os.makedirs(target_dir, exist_ok=True)
    
    for url in tqdm(urls):
        #print(f'Processing URL: {url}')
        
        # Sende eine HTTP GET-Anfrage an die URL
        response = requests.get(url)
        if response.status_code == 200:
            # Extrahiere den Dateinamen aus dem Link
            start_index = url.rfind('/') + 1
            end_index = url.rfind('.zip') + 4  # `.zip` ist 4 Zeichen lang
            zip_filename = url[start_index:end_index]
            
            # Bereinige den Dateinamen
            cleaned_filename = re.sub(r'[^a-zA-Z0-9]', '_', zip_filename)
            zip_file_path = os.path.join(target_dir, cleaned_filename)
            #print(f'Downloading ZIP file: {zip_file_path}')
            
            # Speichere die ZIP-Datei lokal im Zielordner
            with open(zip_file_path, 'wb') as f:
                f.write(response.content)
            #print(f'ZIP-Datei erfolgreich heruntergeladen: {zip_file_path}')
            
            # Extrahiere die Zahlen aus dem Dateinamen
            numbers = re.findall(r'\d+', zip_filename)
            if numbers:
                extracted_numbers = numbers[0]  # Nimm die erste gefundene Zahl
            else:
                extracted_numbers = 'unknown'
            
            # Erstelle den Namen für das entpackte Verzeichnis
            #extract_dir = os.path.join(target_dir, f'jb_JURE{extracted_numbers}_entzip')
            #os.makedirs(extract_dir, exist_ok=True)  # Stelle sicher, dass das Zielverzeichnis existiert
            
            # Entpacke die ZIP-Datei
            unpacked_xmls = "Gerichtsurteile/_xml"
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(unpacked_xmls)
            #print(f'Dateien erfolgreich in {extract_dir} entpackt.')
            
            # Optional: Lösche die ZIP-Datei nach dem Entpacken
            os.remove(zip_file_path)
        else:
            print(f'Fehler beim Herunterladen der ZIP-Datei. Statuscode: {response.status_code}')
            print(url)
        
        time.sleep(1 + random.random()/4)

In [ ]:
download_and_extract_zip(links_list, target_dir)

 53%|█████▎    | 46/87 [00:56<00:50,  1.24s/it]

In [2]:
xmls = [x for x in os.listdir("Gerichtsurteile/_xml") if x[-4:] == ".xml"]

In [8]:
# Gerichtsurteile aus xml Format in txt Dateien uberführen
for i in tqdm(xmls):
    urteil_xml = BeautifulSoup(open("Gerichtsurteile/_xml/"+i,"r"),"xml")
    first_levels = urteil_xml.find("dokument").find_all(recursive=False)
    doc_as_str = ""
    ls = []
    for element in first_levels:
        dl_elements = element.find_all("dl", class_="RspDL")
        for dl in dl_elements:
            if dl.text.strip() != "":
                if "\n\n\n" in dl.text.strip():
                    doc_as_str+= dl.text.strip().split("\n\n\n")[1]+"\n"
                    ls.append(dl.text.strip())
    doc_as_str = doc_as_str.strip()
    gericht = urteil_xml.find("gertyp").text
    os.makedirs(f"Gerichtsurteile/{gericht}/", exist_ok=True)
    new_name = i.replace("xml","txt")
    filename = f"Gerichtsurteile/{gericht}/{new_name}"
    with open(filename, "w") as f:
        f.write(doc_as_str)

100%|██████████| 75561/75561 [20:11<00:00, 62.35it/s]
